In [ ]:
import xml.etree.ElementTree as ET
import json
import re

def safe_float(text):
    """Convert a string to float after removing commas."""
    if text is None:
        return 0.0
    return float(re.sub(r",", "", text.strip()))

def parse_decision_logs(root, hero):
    """
    Parse the XML tree and, for every opponent decision (action) in rounds 1 and later
    (preflop and beyond), produce a snapshot log capturing the game state at the moment
    BEFORE the action is processed.
    
    In this snapshot:
      - The snapshot does NOT include any actions from round 0 (blinds/antes).
      - The re-indexed action counter is incremented for every action in rounds ≥ 1,
        so that no action gets a number of 0.
      - Only opponent actions are logged (the hero's actions are skipped).
      - The snapshot shows the acting opponent's hole cards (if available); the hero's
        or other players' hole cards are not revealed in the snapshot.
    """
    logs = []

    for game in root.findall('game'):
        general = game.find('general')
        if general is None:
            continue

        # --- Extract blinds and ante ---
        small_blind_elem = general.find('smallblind')
        big_blind_elem = general.find('bigblind')
        ante_elem = general.find('ante')
        small_blind = safe_float(small_blind_elem.text) if small_blind_elem is not None else 0.0
        big_blind = safe_float(big_blind_elem.text) if big_blind_elem is not None else 0.0
        ante = safe_float(ante_elem.text) if ante_elem is not None else 0.0

        # --- Process players ---
        players = {}        # key: player name, value: player info
        active_players = {} # tracks whether a player is still in the hand
        players_elem = general.find('players')
        if players_elem is None:
            continue

        for player_elem in players_elem.findall('player'):
            name = player_elem.attrib.get('name')
            seat = int(player_elem.attrib.get('seat'))
            is_dealer = (player_elem.attrib.get('dealer') == "1")
            chips = safe_float(player_elem.attrib.get('chips'))
            bet = safe_float(player_elem.attrib.get('bet'))
            win = safe_float(player_elem.attrib.get('win'))
            players[name] = {
                'seat': seat,
                'is_dealer': is_dealer,
                'chips': chips,
                'bet': bet,
                'win': win
            }
            active_players[name] = True

        # --- Determine relative positions ---
        # Sort players by seat number.
        sorted_players = sorted(players.items(), key=lambda item: item[1]['seat'])
        # Find the dealer (button). If none is marked, assume the first player is the dealer.
        dealer_index = None
        for i, (name, info) in enumerate(sorted_players):
            if info['is_dealer']:
                dealer_index = i
                break
        if dealer_index is None:
            dealer_index = 0

        button_player = sorted_players[dealer_index][0]
        small_blind_player = sorted_players[(dealer_index + 1) % len(sorted_players)][0]
        big_blind_player = sorted_players[(dealer_index + 2) % len(sorted_players)][0]

        # Assign relative position codes: small blind = 0, big blind = 1, button = 2, others = 3.
        player_positions = {}
        for name in players:
            if name == small_blind_player:
                players[name]['position_relative'] = 0
            elif name == big_blind_player:
                players[name]['position_relative'] = 1
            elif name == button_player:
                players[name]['position_relative'] = 2
            else:
                players[name]['position_relative'] = 3
            player_positions[name] = players[name]['position_relative']

        # Skip this game if the hero is not present.
        if hero not in players:
            continue

        # --- Initialize state variables ---
        pot_size = 0.0
        cumulative_actions = []  # will contain actions from rounds >= 1 only
        board_cards = []         # community cards seen so far
        pocket_cards = {}        # updated when encountering <cards type="Pocket">
        snapshot_action_counter = 0  # counter for actions in rounds >= 1

        # --- Process rounds in order (sorted by round number) ---
        rounds = game.findall('round')
        rounds = sorted(rounds, key=lambda r: int(r.attrib.get('no')))
        for r in rounds:
            round_no = int(r.attrib.get('no'))
            for elem in r:
                if elem.tag == 'cards':
                    card_type = elem.attrib.get('type')
                    if card_type == 'Pocket':
                        # Record pocket cards for the given player.
                        player = elem.attrib.get('player')
                        text = elem.text.strip() if elem.text else ""
                        cards = text.split()
                        # If cards are hidden (e.g., "X X"), record as unknown.
                        if any(card.upper().startswith("X") for card in cards):
                            pocket_cards[player] = ["unknown", "unknown"]
                        else:
                            pocket_cards[player] = cards
                    else:
                        # Community cards (flop, turn, river)
                        cards = elem.text.split() if elem.text else []
                        board_cards.extend(cards)
                elif elem.tag == 'action':
                    # Build an action dictionary.
                    action_details = {
                        'round': round_no,
                        'player': elem.attrib.get('player'),
                        'action_type': int(elem.attrib.get('type')),
                        'action_sum': safe_float(elem.attrib.get('sum'))
                    }

                    # For actions in round 0 (blinds/antes), update state only (do not record them).
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        # (If desired, you could update pocket_cards from round 0 as well.)
                        continue

                    # For actions in rounds >= 1, first assign a new re-indexed action number.
                    snapshot_action_counter += 1
                    # Copy the action_details and add the new action number.
                    action_details['action_no'] = snapshot_action_counter

                    # If the acting player is not the hero, produce a snapshot BEFORE processing the action.
                    # (The snapshot’s "previous_actions" will be the cumulative_actions so far.)
                    if action_details['player'] != hero:
                        is_button = (action_details['player'] == button_player)
                        actor_cards = pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        # Build the snapshot.
                        snapshot = {
                            "gamecode": game.attrib.get("gamecode"),
                            "round_no": round_no,
                            "current_street": (
                                "preflop" if round_no == 1 else
                                "flop" if round_no == 2 else
                                "turn" if round_no == 3 else
                                "river" if round_no == 4 else "unknown"
                            ),
                            "blinds": {
                                "small_blind": small_blind,
                                "big_blind": big_blind,
                                "ante": ante
                            },
                            "player_positions": player_positions,
                            "player_stacks": {name: players[name]['chips'] for name in players},
                            "pot_size": pot_size,  # state BEFORE processing this action
                            "board_cards": board_cards.copy(),
                            "previous_actions": cumulative_actions.copy(),  # only round>=1 actions so far
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for active in active_players.values() if active),
                            "is_button": is_button,
                            "actor_hole_cards": pocket_cards.get(action_details['player'], ["unknown", "unknown"])
                        }
                        logs.append(snapshot)

                    # Now update the state: add this action to the cumulative history.
                    cumulative_actions.append(action_details)
                    pot_size += action_details['action_sum']
                    # Mark the actor as inactive if the action is a fold (we assume type==0 means fold).
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False

    return logs

if __name__ == '__main__':
    try:
        # Parse the XML file (adjust 'test.xml' to your filename)
        tree = ET.parse('test.xml')
        root = tree.getroot()

        # Determine the hero's nickname from the session-level <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback if not found

        # Parse opponent decision snapshots (skipping hero actions and round 0 actions)
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


## Improve version of extract data from single iPoker HH xml file

In [2]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        print(json.dumps(logs, indent=4))
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


[
    {
        "gamecode": "10136070425",
        "round_no": 1,
        "current_street": "preflop",
        "blinds": {
            "small_blind": 10.0,
            "big_blind": 20.0,
            "ante": 0.0
        },
        "player_positions": {
            "GGM3M": 0,
            "Bekocabron": 1,
            "mmpq8j6x8": 2
        },
        "player_stacks": {
            "GGM3M": 500.0,
            "Bekocabron": 500.0,
            "mmpq8j6x8": 500.0
        },
        "pot_size": 30.0,
        "board_cards": [],
        "previous_actions": [
            {
                "round": 1,
                "player": "mmpq8j6x8",
                "action_type": 0,
                "action_sum": 0.0,
                "action_no": 1,
                "player_position": 2
            }
        ],
        "action": {
            "round": 1,
            "player": "GGM3M",
            "action_type": 3,
            "action_sum": 10.0,
            "action_no": 2
        },
        "players_remainin

## Extract iPoker HH from single hand history file(Ready for training model)

In [6]:
#!/usr/bin/env python3
import xml.etree.ElementTree as ET
import json, re

def safe_float(text):
    try:
        # remove any commas or other non-digit/dot characters
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    # Search all rounds for a <cards type="Pocket" player="..."> element.
    # If the text contains "X" (hidden cards) we return ["unknown","unknown"].
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    # Process every game
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        # extract blinds and ante (if any)
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        # Process players. We assume the players are listed in a <players> element.
        players_elem = general.find('players')
        player_positions = {}   # will hold each player’s relative position code
        player_stacks = {}      # the chip stacks at the beginning of the hand
        # sort players by their seat number
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        # determine the dealer: use the one whose dealer attribute equals "1" (or if none, the first one)
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        # Create an “order” starting with the player immediately after the dealer (wrapping around).
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        # For instance, assign positions 0,1,...,n-1 in the order determined.
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        # All players start active.
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []  # history of actions (only rounds >= 1)
        snapshot_action_counter = 0

        # Process each round in the game.
        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    # If it is not a Pocket card then we assume community cards.
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    # Skip round 0 actions: update pot only.
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    # For rounds >=1: assign a re-indexed action number.
                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    # For opponent actions (non–hero) produce a snapshot BEFORE processing the action.
                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,  # pot _before_ current action
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            # NEW FIELDS:
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        # Add only previous actions that have been recorded (all from rounds>=1).
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    # Update running state: add this action to cumulative_actions and update pot.
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    # If the action is a fold (action_type == 0) then mark the player as inactive.
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
        # end of game rounds
    return logs

if __name__ == '__main__':
    try:
        tree = ET.parse('test.xml')
        root = tree.getroot()
        # Get the hero name from the session <nickname> element.
        session_general = root.find('general')
        if session_general is not None and session_general.find('nickname') is not None:
            hero = session_general.find('nickname').text.strip()
        else:
            hero = "mmpq8j6x8"  # fallback
        logs = parse_decision_logs(root, hero)
        # Instead of printing, write to a JSON file:
        with open('test_hand.json', 'w') as outfile:
            json.dump(logs, outfile, indent=4)
        print("Data extraction complete. Saved to logs.json")
    except ET.ParseError as e:
        print("Error parsing XML file:", e)


Data extraction complete. Saved to logs.json


## Extract iPoker HH from folder

In [2]:
#!/usr/bin/env python3
import os
import xml.etree.ElementTree as ET
import json
import re

# ------------------------------
# Your helper functions
# ------------------------------

def safe_float(text):
    try:
        return float(re.sub(r"[^\d\.]", "", text))
    except Exception:
        return 0.0

def street_from_round(round_no):
    return {1: "preflop", 2: "flop", 3: "turn", 4: "river"}.get(round_no, "unknown")

def get_hole_cards(game, player):
    for r in game.findall('round'):
        for elem in r.findall('cards'):
            if elem.attrib.get("type") == "Pocket" and elem.attrib.get("player") == player:
                if elem.text:
                    cards = elem.text.split()
                    if any(c.upper().startswith("X") for c in cards):
                        return ["unknown", "unknown"]
                    else:
                        return cards
    return ["unknown", "unknown"]

def parse_decision_logs(root, hero):
    logs = []
    for game in root.findall('game'):
        gamecode = game.attrib.get("gamecode", "")
        general = game.find('general')
        blinds = {}
        blinds['small_blind'] = safe_float(general.findtext('smallblind', default="0"))
        blinds['big_blind'] = safe_float(general.findtext('bigblind', default="0"))
        blinds['ante'] = safe_float(general.findtext('ante', default="0"))
        
        players_elem = general.find('players')
        player_positions = {}
        player_stacks = {}
        sorted_players = sorted(players_elem.findall('player'), key=lambda p: int(p.attrib.get('seat', 0)))
        names = [p.attrib['name'] for p in sorted_players]
        dealer = None
        for p in sorted_players:
            if p.attrib.get('dealer','0')=='1':
                dealer = p.attrib['name']
                break
        if dealer is None:
            dealer = names[0]
        dealer_index = names.index(dealer)
        order = names[dealer_index+1:] + names[:dealer_index+1]
        for i, name in enumerate(order):
            player_positions[name] = i
        for p in sorted_players:
            name = p.attrib['name']
            player_stacks[name] = safe_float(p.attrib.get('chips', "0"))
        
        active_players = {name: True for name in player_positions}
        pot_size = 0.0
        board_cards = []
        cumulative_actions = []
        snapshot_action_counter = 0

        for r in game.findall('round'):
            round_no = int(r.attrib.get('no', 0))
            for child in r:
                if child.tag == "cards":
                    if child.attrib.get("type") != "Pocket":
                        if child.text:
                            board_cards.extend(child.text.split())
                elif child.tag == "action":
                    action_details = {
                        'round': round_no,
                        'player': child.attrib.get('player'),
                        'action_type': int(child.attrib.get('type')),
                        'action_sum': safe_float(child.attrib.get('sum'))
                    }
                    if round_no < 1:
                        pot_size += action_details['action_sum']
                        continue

                    snapshot_action_counter += 1
                    action_details['action_no'] = snapshot_action_counter

                    if child.attrib.get('player') != hero:
                        snapshot = {
                            "gamecode": gamecode,
                            "round_no": round_no,
                            "current_street": street_from_round(round_no),
                            "blinds": blinds,
                            "player_positions": player_positions,
                            "player_stacks": player_stacks,
                            "pot_size": pot_size,
                            "board_cards": board_cards.copy(),
                            "previous_actions": [],
                            "action": action_details.copy(),
                            "players_remaining": sum(1 for v in active_players.values() if v),
                            "is_button": (player_positions.get(child.attrib.get('player')) == len(player_positions)-1),
                            "actor_hole_cards": get_hole_cards(game, child.attrib.get('player')),
                            "actor_stack_size": player_stacks.get(child.attrib.get('player'), 0.0),
                            "actor_position": player_positions.get(child.attrib.get('player'))
                        }
                        for act in cumulative_actions:
                            act2 = act.copy()
                            act2["player_position"] = player_positions.get(act["player"])
                            snapshot["previous_actions"].append(act2)
                        logs.append(snapshot)
                    
                    cumulative_actions.append(action_details.copy())
                    pot_size += action_details['action_sum']
                    if action_details['action_type'] == 0:
                        active_players[action_details['player']] = False
    return logs

# ------------------------------
# Process All XML Files in ipoker_hh Folder
# ------------------------------

def process_all_hand_history(root_folder, hero=None):
    all_logs = []
    # Walk through every subdirectory and file
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for filename in filenames:
            if filename.endswith(".xml"):
                file_path = os.path.join(dirpath, filename)
                try:
                    tree = ET.parse(file_path)
                    root_xml = tree.getroot()
                    # If hero is not provided, try to extract it from the session info
                    if hero is None:
                        session_general = root_xml.find('general')
                        if session_general is not None and session_general.find('nickname') is not None:
                            hero = session_general.find('nickname').text.strip()
                    logs = parse_decision_logs(root_xml, hero)
                    all_logs.extend(logs)
                    print(f"Processed file: {file_path} -> {len(logs)} snapshots.")
                except ET.ParseError as e:
                    print(f"Error parsing XML file: {file_path}", e)
    return all_logs

if __name__ == '__main__':
    # Change the path below to the location of your ipoker_hh folder
    root_folder = "ipoker_hh_test"
    # You can set hero manually or leave it None to try extracting from the XML files.
    hero_name = None  # or "your_hero_name_here"
    
    print("Processing all hand history XML files in folder:", root_folder)
    all_logs = process_all_hand_history(root_folder, hero=hero_name)
    print("Total snapshots extracted:", len(all_logs))
    
    # Save all logs to a JSON file so that the training script can use them.
    with open("logs.json", "w") as outfile:
        json.dump(all_logs, outfile, indent=4)
    print("Data extraction complete. Saved to logs.json")


Processing all hand history XML files in folder: ipoker_hh_test
Processed file: ipoker_hh_test\Tournaments\4967863818.xml -> 3 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967864063.xml -> 32 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967864817.xml -> 51 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967966606.xml -> 33 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967966682.xml -> 31 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967966752.xml -> 42 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967966844.xml -> 64 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967967010.xml -> 4 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967967313.xml -> 67 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967967817.xml -> 53 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967967920.xml -> 34 snapshots.
Processed file: ipoker_hh_test\Tournaments\4967968111.xml -> 59 snapshots.
Processed file: ipoker_hh_test\Tournam

## Exam the output json file

In [4]:
import json

with open('logs.json', 'r') as f:
    data = json.load(f)
print("Number of snapshots:", len(data))
print("First snapshot:")
print(json.dumps(data[1], indent=4))


Number of snapshots: 498975
First snapshot:
{
    "gamecode": "6115617928",
    "round_no": 1,
    "current_street": "preflop",
    "blinds": {
        "small_blind": 1000.0,
        "big_blind": 2000.0,
        "ante": 0.0
    },
    "player_positions": {
        "soleggiata033": 0,
        "terpitors": 1,
        "ZioSergio3674": 2
    },
    "player_stacks": {
        "terpitors": 50000.0,
        "ZioSergio3674": 50000.0,
        "soleggiata033": 50000.0
    },
    "pot_size": 55000.0,
    "board_cards": [],
    "previous_actions": [
        {
            "round": 1,
            "player": "ZioSergio3674",
            "action_type": 3,
            "action_sum": 2000.0,
            "action_no": 1,
            "player_position": 2
        },
        {
            "round": 1,
            "player": "soleggiata033",
            "action_type": 23,
            "action_sum": 50000.0,
            "action_no": 2,
            "player_position": 0
        }
    ],
    "action": {
        "round

## Model training

In [5]:
#!/usr/bin/env python3
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input, Dense, LSTM, Embedding, Concatenate, Dropout
)
from tensorflow.keras.optimizers import Adam

# =============================================================================
# 1. Helper Functions for Feature Encoding
# =============================================================================

def one_hot_round(round_no):
    """One-hot encode round number (1=preflop, 2=flop, 3=turn, 4=river)."""
    vec = np.zeros(4)
    if 1 <= round_no <= 4:
        vec[round_no - 1] = 1
    return vec

def one_hot_position(pos, max_players=10):
    """One-hot encode a player's position (an integer in [0, max_players-1])."""
    vec = np.zeros(max_players)
    if pos < max_players:
        vec[pos] = 1
    return vec

def card_to_onehot(card):
    """
    Convert a card string (e.g., 'S4', 'HA', 'C10') to a 52-dim one-hot vector.
    If the card is hidden (e.g., starts with 'X') it returns an all-zeros vector.
    """
    ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
    suits = ['S','H','D','C']
    onehot = np.zeros(52)
    if card is None or card.upper().startswith("X"):
        return onehot
    suit = card[0]
    rank = card[1:]
    if suit in suits and rank in ranks:
        suit_index = suits.index(suit)
        rank_index = ranks.index(rank)
        index = suit_index * 13 + rank_index
        onehot[index] = 1
    return onehot

def encode_board_cards(board_cards, max_cards=5):
    """
    Encode the board cards as the concatenation of one-hot vectors (52 dims each).
    Pads with zeros if there are fewer than max_cards.
    """
    encoded = []
    for card in board_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < max_cards:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:max_cards])

def encode_hole_cards(hole_cards):
    """
    Encode the player's (or actor's) hole cards (expected to be a list of 2 cards)
    as a concatenation of two 52-dim one-hot vectors.
    """
    encoded = []
    for card in hole_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < 2:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:2])

# =============================================================================
# 2. Process Each Snapshot into Model Inputs and Target
# =============================================================================

def process_snapshot(snapshot):
    """
    From a snapshot dictionary, create:
      - A vector of static features
      - A sequence of previous actions (each with an action type and amount)
      - The target (opponent's current action type)
      
    **Static features include:**
      - One-hot encoded round (4 dims)
      - Pot size (1 dim; scaled)
      - Blinds: small, big, ante (3 dims; scaled)
      - Actor stack (1 dim; scaled)
      - Actor position (one-hot, 10 dims)
      - Number of players remaining (1 dim; scaled)
      - Board cards (5 fixed cards × 52 dims = 260 dims)
      - Actor hole cards (2 cards × 52 dims = 104 dims)
      
    **Sequential features:**  
      For each previous action (up to a fixed max length) we use:
      - action_type (integer; offset by +1 so that 0 is reserved for padding)
      - action_sum (float; scaled)
    """
    # --- Static features ---
    # 1. Round (from current action)
    round_no = int(snapshot["action"]["round"])
    round_vec = one_hot_round(round_no)
    
    # 2. Pot size (scale by 100)
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    
    # 3. Blinds and ante (scaled)
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    
    # 4. Actor stack size (scale by 1000)
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    
    # 5. Actor position (one-hot with dimension 10)
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    
    # 6. Number of players remaining (scale by 10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # 7. Board cards (5 fixed cards)
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # 8. Actor hole cards (2 cards)
    hole_cards_vec = encode_hole_cards(snapshot.get("actor_hole_cards", []))
    
    # Concatenate all static features:
    # Total dims: 4 + 1 + 3 + 1 + 10 + 1 + 260 + 104 = 384
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec, hole_cards_vec
    ])
    
    # --- Sequential features ---
    # For each previous action, we take:
    #   - action_type (offset by +1 so that 0 is our pad value)
    #   - action_sum (scaled by 100)
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        act_type = int(action.get("action_type", 0)) + 1  # reserve 0 for padding
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20  # maximum number of previous actions to consider
    # Truncate if too long
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    # Pad sequences (pad type=0, which for action_type will be masked in the Embedding layer)
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # --- Target: Opponent's current action type (as integer) ---
    target = int(snapshot["action"].get("action_type", 0))
    
    return static_features, seq_types, seq_amounts, target

# =============================================================================
# 3. Load and Preprocess Data
# =============================================================================

def load_and_preprocess_data(json_filename):
    """
    Load snapshot logs from a JSON file and create training arrays.
    The JSON file is expected to be a list of snapshots.
    """
    with open(json_filename, 'r') as f:
        data = json.load(f)
    
    static_features_list = []
    seq_types_list = []
    seq_amounts_list = []
    targets = []
    
    for snapshot in data:
        static_feat, seq_types, seq_amounts, target = process_snapshot(snapshot)
        static_features_list.append(static_feat)
        seq_types_list.append(seq_types)
        seq_amounts_list.append(seq_amounts)
        targets.append(target)
    
    X_static = np.stack(static_features_list)        # shape: (N, 384)
    X_seq_types = np.stack(seq_types_list)             # shape: (N, MAX_SEQ_LENGTH)
    X_seq_amounts = np.stack(seq_amounts_list)         # shape: (N, MAX_SEQ_LENGTH, 1)
    y = np.array(targets, dtype=np.int32)              # shape: (N,)
    
    return X_static, X_seq_types, X_seq_amounts, y

# Change the filename below to your JSON file produced by your XML parser.
JSON_FILENAME = 'logs.json'
X_static, X_seq_types, X_seq_amounts, y = load_and_preprocess_data(JSON_FILENAME)

# (Optional) Check the shapes of your training arrays:
print("X_static shape:", X_static.shape)
print("X_seq_types shape:", X_seq_types.shape)
print("X_seq_amounts shape:", X_seq_amounts.shape)
print("y shape:", y.shape)

# Split into training and validation sets
X_static_train, X_static_val, X_seq_types_train, X_seq_types_val, X_seq_amounts_train, X_seq_amounts_val, y_train, y_val = train_test_split(
    X_static, X_seq_types, X_seq_amounts, y, test_size=0.2, random_state=42
)

# =============================================================================
# 4. Build the Keras Model
# =============================================================================

# Parameters for the sequential branch
MAX_SEQ_LENGTH = 20
# Adjust NUM_ACTION_TYPES based on your data (here we assume 25; update if needed)
NUM_ACTION_TYPES = 25  
EMBEDDING_DIM = 16  
NUM_ACTION_CLASSES = 30  # number of distinct action types to predict (update as needed)

# -- Static Input Branch --
static_input = Input(shape=(384,), name='static_input')
x_static = Dense(128, activation='relu')(static_input)
x_static = Dense(64, activation='relu')(x_static)

# -- Sequential Input Branch --
# Input for action types (integers; shape = (MAX_SEQ_LENGTH,))
seq_types_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='seq_types_input')
# Input for action amounts (floats; shape = (MAX_SEQ_LENGTH, 1))
seq_amounts_input = Input(shape=(MAX_SEQ_LENGTH, 1), dtype='float32', name='seq_amounts_input')

# Process action types with an Embedding layer.
# (We use mask_zero=True so that padded 0 values are ignored.)
x_seq_types = Embedding(
    input_dim=NUM_ACTION_TYPES + 1,  # +1 to reserve index 0 for padding
    output_dim=EMBEDDING_DIM,
    mask_zero=True,
    name='action_type_embedding'
)(seq_types_input)

# Process the amounts with a simple dense layer (applied to each time step).
x_seq_amounts = Dense(8, activation='relu', name='amount_dense')(seq_amounts_input)

# Concatenate along the feature dimension: now each time step has (EMBEDDING_DIM + 8) features.
x_seq = Concatenate(name='seq_concat')([x_seq_types, x_seq_amounts])

# Process the concatenated sequence with an LSTM.
x_seq = LSTM(64, name='lstm_seq')(x_seq)

# -- Merge Both Branches --
x = Concatenate(name='merge')([x_static, x_seq])
x = Dense(64, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(NUM_ACTION_CLASSES, activation='softmax', name='output')(x)

model = Model(
    inputs=[static_input, seq_types_input, seq_amounts_input],
    outputs=output
)

model.compile(
    optimizer=Adam(learning_rate=1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

# =============================================================================
# 5. Train the Model
# =============================================================================

history = model.fit(
    x={
        'static_input': X_static_train,
        'seq_types_input': X_seq_types_train,
        'seq_amounts_input': X_seq_amounts_train
    },
    y=y_train,
    validation_data=(
        {
            'static_input': X_static_val,
            'seq_types_input': X_seq_types_val,
            'seq_amounts_input': X_seq_amounts_val
        },
        y_val
    ),
    epochs=20,
    batch_size=32
)

# =============================================================================
# 6. Evaluate / Save the Model
# =============================================================================

loss, acc = model.evaluate(
    x={
        'static_input': X_static_val,
        'seq_types_input': X_seq_types_val,
        'seq_amounts_input': X_seq_amounts_val
    },
    y=y_val
)
print(f"Validation Loss: {loss:.4f}  |  Validation Accuracy: {acc:.4f}")

# Optionally, save your model:
model.save("opponent_model.h5")


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

X_static shape: (498975, 384)
X_seq_types shape: (498975, 20)
X_seq_amounts shape: (498975, 20, 1)
y shape: (498975,)
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
seq_types_input (InputLayer)    [(None, 20)]         0                                            
__________________________________________________________________________________________________
seq_amounts_input (InputLayer)  [(None, 20, 1)]      0                                            
__________________________________________

## Testing the Model

In [1]:
#!/usr/bin/env python3
import json
import numpy as np
from tensorflow.keras.models import load_model

# ---------------------------------------------------------------------------
# Helper Functions (same as used during training)
# ---------------------------------------------------------------------------

def one_hot_round(round_no):
    """One-hot encode round number (1=preflop, 2=flop, 3=turn, 4=river)."""
    vec = np.zeros(4)
    if 1 <= round_no <= 4:
        vec[round_no - 1] = 1
    return vec

def one_hot_position(pos, max_players=10):
    """One-hot encode a player's position (an integer in [0, max_players-1])."""
    vec = np.zeros(max_players)
    if pos < max_players:
        vec[pos] = 1
    return vec

def card_to_onehot(card):
    """
    Convert a card string (e.g., 'S4', 'HA', 'C10') to a 52-dimensional one-hot vector.
    If the card is hidden (e.g., starts with 'X') or missing, return a zero vector.
    """
    ranks = ['A','2','3','4','5','6','7','8','9','10','J','Q','K']
    suits = ['S','H','D','C']
    onehot = np.zeros(52)
    if card is None or card.upper().startswith("X"):
        return onehot
    suit = card[0]
    rank = card[1:]
    if suit in suits and rank in ranks:
        suit_index = suits.index(suit)
        rank_index = ranks.index(rank)
        index = suit_index * 13 + rank_index
        onehot[index] = 1
    return onehot

def encode_board_cards(board_cards, max_cards=5):
    """
    Encode the board cards as the concatenation of one-hot vectors (each 52 dimensions).
    Pads with zeros if there are fewer than max_cards.
    """
    encoded = []
    for card in board_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < max_cards:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:max_cards])

def encode_hole_cards(hole_cards):
    """
    Encode the actor's hole cards (a list of 2 cards) as a concatenation of two 52-dim one-hot vectors.
    Pads with zeros if needed.
    """
    encoded = []
    for card in hole_cards:
        encoded.append(card_to_onehot(card))
    while len(encoded) < 2:
        encoded.append(np.zeros(52))
    return np.concatenate(encoded[:2])

def process_snapshot(snapshot):
    """
    Convert a snapshot dictionary into inputs for the model.
      - Static features (384-dimensional vector)
      - Sequential features: two sequences for action types and action amounts (length=20)
      - The target action type (as integer)
    """
    # ----- Static features -----
    round_no = int(snapshot["action"]["round"])
    round_vec = one_hot_round(round_no)
    
    # Scale pot size by 100
    pot_size = np.array([float(snapshot["pot_size"]) / 100.0])
    
    # Blinds and ante (scaled)
    blinds = snapshot.get("blinds", {})
    small_blind = float(blinds.get("small_blind", 0)) / 100.0
    big_blind = float(blinds.get("big_blind", 0)) / 100.0
    ante = float(blinds.get("ante", 0)) / 100.0
    blinds_vec = np.array([small_blind, big_blind, ante])
    
    # Actor stack (scaled by 1000)
    actor_stack = np.array([float(snapshot.get("actor_stack_size", 0)) / 1000.0])
    
    # Actor position as one-hot (dimension 10)
    actor_pos = int(snapshot.get("actor_position", 0))
    pos_vec = one_hot_position(actor_pos, max_players=10)
    
    # Number of players remaining (scaled by 10)
    players_remaining = np.array([float(snapshot.get("players_remaining", 0)) / 10.0])
    
    # Board cards: fixed 5 cards (each 52 dimensions)
    board_vec = encode_board_cards(snapshot.get("board_cards", []), max_cards=5)
    
    # Actor hole cards: 2 cards (each 52 dimensions)
    hole_cards_vec = encode_hole_cards(snapshot.get("actor_hole_cards", []))
    
    # Concatenate static features (4 + 1 + 3 + 1 + 10 + 1 + 260 + 104 = 384 dims)
    static_features = np.concatenate([
        round_vec, pot_size, blinds_vec, actor_stack, pos_vec,
        players_remaining, board_vec, hole_cards_vec
    ])
    
    # ----- Sequential features -----
    seq_actions = snapshot.get("previous_actions", [])
    seq_types = []
    seq_amounts = []
    for action in seq_actions:
        # Offset action type by +1 so that 0 is reserved for padding
        act_type = int(action.get("action_type", 0)) + 1  
        act_sum = float(action.get("action_sum", 0)) / 100.0
        seq_types.append(act_type)
        seq_amounts.append(act_sum)
    
    MAX_SEQ_LENGTH = 20
    seq_types = seq_types[:MAX_SEQ_LENGTH]
    seq_amounts = seq_amounts[:MAX_SEQ_LENGTH]
    # Pad the sequences if needed
    while len(seq_types) < MAX_SEQ_LENGTH:
        seq_types.append(0)
        seq_amounts.append(0.0)
    seq_types = np.array(seq_types, dtype=np.int32)
    seq_amounts = np.array(seq_amounts, dtype=np.float32).reshape((MAX_SEQ_LENGTH, 1))
    
    # ----- Target -----
    target = int(snapshot["action"].get("action_type", 0))
    
    return static_features, seq_types, seq_amounts, target

# ---------------------------------------------------------------------------
# Load the trained model
# ---------------------------------------------------------------------------

model = load_model("opponent_model.h5")
print("Model loaded successfully.")

# ---------------------------------------------------------------------------
# Load a test snapshot from logs.json
# ---------------------------------------------------------------------------

# Open your JSON file that contains the extracted snapshots
with open("test_hand.json", "r") as f:
    snapshots = json.load(f)

# For testing, we'll use the first snapshot in the file.
test_snapshot = snapshots[0]
print("Testing on the first snapshot from logs.json:")

# Optionally, print the snapshot to inspect it:
import pprint
pprint.pprint(test_snapshot)

# ---------------------------------------------------------------------------
# Process the test snapshot into model inputs
# ---------------------------------------------------------------------------

static_features, seq_types, seq_amounts, true_target = process_snapshot(test_snapshot)

# Reshape to add the batch dimension (for a single sample)
static_features = np.expand_dims(static_features, axis=0)    # Shape: (1, 384)
seq_types = np.expand_dims(seq_types, axis=0)                  # Shape: (1, 20)
seq_amounts = np.expand_dims(seq_amounts, axis=0)              # Shape: (1, 20, 1)

# ---------------------------------------------------------------------------
# Make a prediction using the model
# ---------------------------------------------------------------------------

predictions = model.predict({
    "static_input": static_features,
    "seq_types_input": seq_types,
    "seq_amounts_input": seq_amounts
})

# The output is a probability vector for each of the 30 action classes.
predicted_class = np.argmax(predictions, axis=1)[0]

print("\n--- Prediction Results ---")
print("Predicted action type:", predicted_class)
print("True action type (from snapshot):", true_target)
print("Prediction probabilities:\n", predictions[0])


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorflow\python\framework\dtypes.py:519: 

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\Users\j4829\.conda\envs\tf\lib\site-packages\tensorboard\compat\ten

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model loaded successfully.
Testing on the first snapshot from logs.json:
{'action': {'action_no': 2,
            'action_sum': 10.0,
            'action_type': 3,
            'player': 'GGM3M',
            'round': 1},
 'actor_hole_cards': ['C10', 'D5'],
 'actor_position': 0,
 'actor_stack_size': 500.0,
 'blinds': {'ante': 0.0, 'big_blind': 20.0, 'small_blind': 10.0},
 'board_cards': [],
 'current_street': 'preflop',
 'gamecode': '10136070425',
 'is_button': False,
 'player_positions': {'Bekocabron': 1, 'GGM3M': 0, 'mmpq8j6x8': 2},
 'player_stacks': {'Bekocabron': 500.0, 'GGM3M': 500.0, 'mmpq8j6x8': 500.0},
 'players_remaining': 2,
 'pot_size': 30.0,
 'previous_actions': [{'action_no': 1,
                       'action_sum': 0.0,
                       'action_type': 0,
                       'player': 'mmpq8j6x8',
                       'player_position': 2,
                       'round': 1}